In [2]:
#loading the modules required 
import torch
from torch import nn
from torchvision import datasets, transforms,models 
from glob import glob
import cv2
import os 
from PIL import Image
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [6]:
'''
use transform  to make changes to the images we need load for classification


'''
transform = transforms.Compose([
 transforms.Resize((256,256)),                
 transforms.ToTensor(),   
 transforms.Normalize(                                                
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])


In [7]:
#specify path to the train,val and test folder 
path_train="chest_xray/train"
path_validation="chest_xray/val"
path_test="chest_xray/test"

In [8]:
"""
loading images for classification  we create folder for each class images n using dataset.
ImageFolder the labels are directly assigned.

"""
train_data = datasets.ImageFolder(path_train,       
                    transform=transform)
validation_data = datasets.ImageFolder(path_validation,       
                    transform=transform)
test_data=datasets.ImageFolder(path_test,       
                    transform=transform)

In [9]:
#DataLoader function to load the data in pytorch format  
trainloader=torch.utils.data.DataLoader(dataset=train_data,shuffle=True,batch_size=32)
validationloader=torch.utils.data.DataLoader(dataset=validation_data,shuffle=True,batch_size=32)
test_loader=torch.utils.data.DataLoader(dataset=test_data,shuffle=True,batch_size=32)

In [7]:
#laod the resnet34 model with pretrained weights
model = models.resnet34(pretrained=True)

In [8]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
#use this no to train the model parameters again
for param in model.parameters():
    param.requires_grad = False

#add the last linear layer for custom classification     
model.fc = nn.Sequential(nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))
#set the loss function
criterion = nn.NLLLoss()

#set the optimizer to be used 
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.005)

#model to device  
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
epochs = 10    #number of epochs 
steps = 0
running_loss = 0
print_every = 10     #used for printing different verbose 
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:     
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()     #initiate optimizer 
        logps = model.forward(inputs) 
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in validationloader:
                    inputs, labels = inputs.to(device),labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(validationloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(validationloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(validationloader):.3f}")
            running_loss = 0
            model.train()
    torch.save(model, 'models/pneumonia_prediction_1 '+str(epoch)+' .pth')

Epoch 1/10.. Train loss: 0.281.. Test loss: 0.455.. Test accuracy: 0.625
Epoch 1/10.. Train loss: 0.197.. Test loss: 1.233.. Test accuracy: 0.500
Epoch 1/10.. Train loss: 0.203.. Test loss: 1.051.. Test accuracy: 0.562
Epoch 1/10.. Train loss: 0.212.. Test loss: 0.476.. Test accuracy: 0.625
Epoch 1/10.. Train loss: 0.241.. Test loss: 0.336.. Test accuracy: 0.750
Epoch 1/10.. Train loss: 0.295.. Test loss: 0.296.. Test accuracy: 0.938
Epoch 1/10.. Train loss: 0.217.. Test loss: 0.789.. Test accuracy: 0.562
Epoch 1/10.. Train loss: 0.188.. Test loss: 0.957.. Test accuracy: 0.562
Epoch 1/10.. Train loss: 0.136.. Test loss: 0.230.. Test accuracy: 0.938
Epoch 1/10.. Train loss: 0.196.. Test loss: 0.354.. Test accuracy: 0.750
Epoch 1/10.. Train loss: 0.186.. Test loss: 0.308.. Test accuracy: 0.750
Epoch 1/10.. Train loss: 0.147.. Test loss: 0.678.. Test accuracy: 0.625
Epoch 1/10.. Train loss: 0.161.. Test loss: 0.609.. Test accuracy: 0.625
Epoch 1/10.. Train loss: 0.161.. Test loss: 0.326..

C:\Users\niraj\Anaconda3\envs\yourenvname\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\niraj\Anaconda3\envs\yourenvname\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\niraj\Anaconda3\envs\yourenvname\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\niraj\Anaconda3\envs\yourenvname\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loadin

Epoch 2/10.. Train loss: 0.149.. Test loss: 1.295.. Test accuracy: 0.562
Epoch 2/10.. Train loss: 0.256.. Test loss: 0.657.. Test accuracy: 0.562
Epoch 2/10.. Train loss: 0.174.. Test loss: 0.216.. Test accuracy: 0.938
Epoch 2/10.. Train loss: 0.145.. Test loss: 0.874.. Test accuracy: 0.562
Epoch 2/10.. Train loss: 0.195.. Test loss: 0.704.. Test accuracy: 0.562
Epoch 2/10.. Train loss: 0.158.. Test loss: 0.385.. Test accuracy: 0.812
Epoch 2/10.. Train loss: 0.126.. Test loss: 0.311.. Test accuracy: 0.750
Epoch 2/10.. Train loss: 0.129.. Test loss: 0.297.. Test accuracy: 0.812
Epoch 2/10.. Train loss: 0.130.. Test loss: 0.183.. Test accuracy: 0.938
Epoch 2/10.. Train loss: 0.155.. Test loss: 1.167.. Test accuracy: 0.562
Epoch 2/10.. Train loss: 0.165.. Test loss: 0.688.. Test accuracy: 0.562
Epoch 2/10.. Train loss: 0.118.. Test loss: 0.181.. Test accuracy: 0.938
Epoch 2/10.. Train loss: 0.156.. Test loss: 0.609.. Test accuracy: 0.625
Epoch 2/10.. Train loss: 0.146.. Test loss: 0.951..

Epoch 8/10.. Train loss: 0.105.. Test loss: 0.306.. Test accuracy: 0.812
Epoch 9/10.. Train loss: 0.079.. Test loss: 0.272.. Test accuracy: 0.812
Epoch 9/10.. Train loss: 0.076.. Test loss: 0.101.. Test accuracy: 1.000
Epoch 9/10.. Train loss: 0.146.. Test loss: 0.259.. Test accuracy: 0.812
Epoch 9/10.. Train loss: 0.134.. Test loss: 0.343.. Test accuracy: 0.812
Epoch 9/10.. Train loss: 0.111.. Test loss: 0.332.. Test accuracy: 0.812
Epoch 9/10.. Train loss: 0.173.. Test loss: 0.197.. Test accuracy: 0.938
Epoch 9/10.. Train loss: 0.107.. Test loss: 0.299.. Test accuracy: 0.750
Epoch 9/10.. Train loss: 0.134.. Test loss: 0.100.. Test accuracy: 1.000
Epoch 9/10.. Train loss: 0.102.. Test loss: 0.269.. Test accuracy: 0.875
Epoch 9/10.. Train loss: 0.090.. Test loss: 0.131.. Test accuracy: 1.000
Epoch 9/10.. Train loss: 0.079.. Test loss: 0.365.. Test accuracy: 0.750
Epoch 9/10.. Train loss: 0.080.. Test loss: 0.157.. Test accuracy: 0.938
Epoch 9/10.. Train loss: 0.193.. Test loss: 0.259..

In [11]:
#testing the model on test dataset available 
model1 = torch.load("models/pneumonia_prediction_1 9 .pth")
accuracy=0 
model1.eval()
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_loader):
        # evaluate the model on the test set
        inputs, labels = inputs.to(device),targets.to(device)
        yhat = model1.forward(inputs)
        ps = torch.exp(yhat)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
print(f"Test accuracy: {(accuracy/len(test_loader) )*100}")

Test accuracy: 85.0
